<a href="https://colab.research.google.com/github/xBrymer/COVID19-AI-CT-Scan-Detection/blob/master/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [13]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_hub as hub

import os
import tqdm
from os.path import join
from zipfile import ZipFile

from tensorflow.keras.models import Sequential, load_model, save_model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.callbacks import EarlyStopping



In [4]:
ds_path = '/content/drive/My Drive/External Datasets/COVID19-Project/Datasets/CT-Scan-split.zip'

f =  ZipFile(ds_path, 'r')
f.extractall('/content')

In [5]:
img_height = 512
img_width = 512

batch_size = 64

test_dir = '/content/main/testing'
train_dir = '/content/main/training'
val_dir = '/content/main/validation'

In [10]:
datagen = ImageDataGenerator(rescale = 1/255)

train_gen = datagen.flow_from_directory(
    directory = train_dir,
    shuffle = True,
    target_size = (img_height, img_width),
    batch_size = 64,
    classes = ['covid', 'non-covid', 'non-informative']
)

val_gen = datagen.flow_from_directory(
    directory = val_dir,
    shuffle = True,
    target_size = (img_height, img_width),
    batch_size = 64,
    classes = ['covid', 'non-covid', 'non-informative']
)

test_gen = datagen.flow_from_directory(
    directory = test_dir,
    shuffle = True,
    target_size = (img_height, img_width),
    batch_size = 64,
    classes = ['covid', 'non-covid', 'non-informative']
)



Found 11834 images belonging to 3 classes.
Found 3771 images belonging to 3 classes.
Found 1884 images belonging to 3 classes.


In [11]:
pretrained_model = tf.keras.applications.Xception(input_shape =(512,512,3), weights = 'imagenet', include_top = False)
pretrained_model.trainable = False

83689472/83683744 [==============================] - 1s 0us/step


In [25]:
model = Sequential()
model.add(pretrained_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(.20))
model.add(Dense(3, activation = 'softmax' ))

In [32]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adamax(),
              metrics = ['accuracy'])

In [38]:
model.evaluate(test_gen)

30/30 [==============================] - 42s 1s/step - loss: 0.1898 - accuracy: 0.9220


[0.18975605070590973, 0.9219745397567749]

In [31]:
for layer in pretrained_model.layers:
  if 'block14' in layer.name:
    layer.trainable = True

In [36]:
hist = model.fit(train_gen, validation_data= val_gen, epochs = 3,
                 callbacks = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience =1))

Epoch 1/3
185/185 [==============================] - 341s 2s/step - loss: 0.2005 - accuracy: 0.9229 - val_loss: 0.1386 - val_accuracy: 0.9509
Epoch 2/3
185/185 [==============================] - 338s 2s/step - loss: 0.1604 - accuracy: 0.9408 - val_loss: 0.1781 - val_accuracy: 0.9308


In [37]:
os.chdir('/content/drive/My Drive/External Datasets/COVID19-Project/successful_models')
model.save('Xception-v1.h5')